In [1]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('chest_xray/test',
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical', 
                                            shuffle=False)  # Ensure that it doesn't shuffle the data

Found 624 images belonging to 2 classes.


In [3]:
vgg = VGG16(input_shape=[224,224] + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [4]:
model.load_weights('training_1/cp.ckpt')

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
test_loss, test_accuracy = model.evaluate(test_set, steps=len(test_set), batch_size=8)

NotFoundError: Graph execution error:

Detected at node 'model/block1_conv1/Conv2D' defined at (most recent call last):
    File "c:\Users\Biki\anaconda3\envs\venv1\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Biki\anaconda3\envs\venv1\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Biki\anaconda3\envs\venv1\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Biki\anaconda3\envs\venv1\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\Biki\anaconda3\envs\venv1\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Biki\anaconda3\envs\venv1\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Biki\anaconda3\envs\venv1\lib\asyncio\b

In [ ]:
predictions = model.predict(test_set, steps=len(test_set))

In [ ]:
true_labels = test_set.classes


In [ ]:
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
report = classification_report(true_labels, predicted_labels, target_names=test_set.class_indices.keys())
confusion = confusion_matrix(true_labels, predicted_labels)

# Print the test accuracy and other metrics
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
print('\nClassification Report:\n', report)
print('\nConfusion Matrix:\n', confusion)